# Nitrate and Phosphorus Loads from Illinois Rivers
To help enable reproducible cutting-edge science, data and workflows used in Illinois nutrient monitoring will be publically available. This notebook demonstrates the basic concepts of
1. making all data available from the cloud in analysis ready formats;
2. running notebooks locally or in the cloud to generate reports, run models, or create interactive web applets; and
3. enabling anyone to share, run, and modify notebooks using only a web browser.

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/illinois-nutrient-monitoring/biennial-report/blob/master/notebooks/01_report_plots.ipynb)

## Environment Setup

In [ ]:
#%%capture
#!pip install https://github.com/aleaf/Figures/archive/master.zip
#!pip install git+https://github.com/illinois-nutrient-monitoring/biennial-report.git

#!pip install -e /home/thodson/Desktop/Projects/illinois-nutrient-monitoring/biennial-report

In [ ]:
%%capture
import matplotlib.pyplot as plt
from Figures import ReportFigures

%matplotlib inline
rf = ReportFigures()
rf.set_style()

# configure fonts for presentation
import matplotlib.pyplot as plt

In [ ]:
# run for in-notebook presentation
SMALL_SIZE = 10
MEDIUM_SIZE = SMALL_SIZE + 2
BIGGER_SIZE = MEDIUM_SIZE + 2

plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
plt.rc('axes', titlesize=SMALL_SIZE)     # fontsize of the axes title
plt.rc('axes', labelsize=MEDIUM_SIZE)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=SMALL_SIZE)    # legend fontsize
plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title

save_figures = True
figure_count = 0
# set figure dimensions
fig_w = 7.16 #5.51181
fig_h = 4.5

In [ ]:
# load libraries
%load_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np
import json
import xarray as xr
import pint_xarray
from titlecase import titlecase

import warnings
warnings.filterwarnings("ignore", message="the unit of the quantity is stripped when downcasting to ndarray")

# load helper functions
from biennial_report.plot import *
from biennial_report.loads import *

### Configuration

In [ ]:
mass_unit = 'pound'
area_unit = 'acre'
flow_unit = 'cubic feet per second'
nested = False #subtract nested gages

## define study period
baseline_years = np.arange(1980,1996+1)
study_years = np.arange(2017,2021+1)

## Load Datasets

In [ ]:
supergage_metadata = '../data/supergage_network.json'
ambient_metadata = '../data/ambient_network.json'

with open(supergage_metadata) as f:
    supergage_network = json.load(f)

with open(ambient_metadata) as f:
    ambient_network = json.load(f)

supergage_ds = xr.load_dataset('../data/illinois_supergage_annual_loads_continuous.nc').pint.quantify()
ambient_ds = xr.load_dataset('../data/illinois_ambient_annual_loads_wrtdsk.nc').pint.quantify()

labels(supergage_network, include_nested=nested)

### Convert Units

In [ ]:
ambient_conversion = {}
for varname, da in ambient_ds.data_vars.items():
    if da.pint.units == 'kilogram':
        ambient_conversion[varname] = mass_unit
    elif da.pint.units == 'meter ** 3 / second':
        ambient_conversion[varname] = flow_unit

#ambient_conversion
ambient_ds = ambient_ds.pint.to(ambient_conversion)

supergage_ds = supergage_ds.pint.to({'nitrate nitrogen':mass_unit, 'total phosphorus':mass_unit})

supergage_ds

### Fill in missing data

In [ ]:
# Add rock at rockton
rockton_id = '05437500'
rockton = ambient_ds.sel(site=rockton_id)[['nitrate nitrogen', 'total phosphorus']]
supergage_ds = xr.concat([supergage_ds, rockton], dim='site', join='inner')

# TODO fill in missing years or other state-border stations

### Compute drainage areas

In [ ]:
from dataretrieval import nwis
site_df, _ = nwis.get_info(sites=gages(ambient_network, include_nested=True))
site_df = site_df[['site_no','dec_lat_va','dec_long_va','drain_area_va']]
site_ds = site_df.rename({'site_no':'site'}, axis=1).set_index('site').to_xarray()
site_ds = site_ds.pint.quantify({'drain_area_va':'square miles', 'dec_lat_va':'degrees', 'dec_long_va':'degrees'}).pint.to(drain_area_va=area_unit)

network = compute_network_loads(site_ds, ambient_network, subtract_nested=True)
network

### Compute loads from monitoring network

In [ ]:
gages(ambient_network, include_nested=False)

In [ ]:
# scale loads based on state's contributing drainage
# nested=True, so subtract off Rock at Rockton
#ambient_gages = gages(ambient_network, include_nested=False)
#supergage_gages = gages(supergage_network, include_nested=False)

supergage_loads = compute_network_loads(supergage_ds, supergage_network, subtract_nested=True)
ambient_loads = compute_network_loads(ambient_ds, ambient_network, subtract_nested=True)

#ambient_loads = compute_network_loads(ambient_ds.sel(site=gages(ambient_network, include_nested=False)),
#                                      ambient_network, nested=True)

ambient_loads = ambient_loads.where(ambient_loads.year.dt.year >= baseline_years[0]).dropna(dim='year', how='all') # awkward
baseline_mean_load = ambient_loads.sum(dim='river').sel(year=ambient_loads.year.dt.year.isin(baseline_years)).mean()
current_mean_load = ambient_loads.sum(dim='river').sel(year=ambient_loads.year.dt.year.isin(study_years)).mean()

In [ ]:
#supergage_loads.mean(dim='year')

## Methods
Replicates methods from previous biennial reports:
- baseline period: water years 1980–1996 
- current period: water years 2017–2022
- subtracts load for Rock River at Rockton
- rescales other rivers based on their drainage area within the State

but adds
- continuous water quality data


## Results
### Statewide Nitrate Load

In [ ]:
supergage_total_yield = supergage_loads.sum(dim='river')/network['drain_area_va'].sum()
ambient_total_yield = ambient_loads.sum(dim='river')/network['drain_area_va'].sum()


baseline_mean_yield = ambient_total_yield.sel(year=ambient_loads.year.dt.year.isin(baseline_years)).mean()

In [ ]:
parameter='nitrate nitrogen'
loc='upper left'

fig, ax = plt.subplots(figsize=(fig_w, fig_h))


running_average_plot(ds1=ambient_total_yield[parameter].assign_attrs({'label':'Illinois EPA Ambient'}),
                     period1=baseline_years,
                     period2=study_years,
                     ds2=supergage_total_yield[parameter].assign_attrs({'label':'USGS Continuous'}),
                     loc=loc,
                     ax=ax)

ax2 = percentage_scale(baseline_mean_yield[parameter].values, ax)

ax2.set_ylabel('[percentage above baseline]'.capitalize())
text = ax.set_ylabel(f'{parameter}\n[pounds per acre]'.capitalize())

ax.tick_params(bottom=True, top=True, left=True, right=False)
handles, labels = ax.get_legend_handles_labels()
rf.legend(ax, handles, labels, bbox_to_anchor=(1.2, 0))

fig.tight_layout()

if save_figures:
    fig_str = str(figure_count).rjust(2,'0')
    fig.savefig(f'../figures/{fig_str}_annual_{parameter}_load.png', dpi=150)
    figure_count += 1

### Statewide Phosphorus Load

In [ ]:
parameter='total phosphorus'
loc='upper left'

loc='upper left'

fig, ax = plt.subplots(figsize=(fig_w, fig_h))


running_average_plot(ds1=ambient_total_yield[parameter].assign_attrs({'label':'Illinois EPA Ambient'}),
                     period1=baseline_years,
                     period2=study_years,
                     ds2=supergage_total_yield[parameter].assign_attrs({'label':'USGS Continuous'}), #comment this for ambient only
                     loc=loc,
                     ax=ax)

ax2 = percentage_scale(baseline_mean_yield[parameter].values, ax)

ax2.set_ylabel('[percentage above baseline]'.capitalize())
text = ax.set_ylabel(f'{parameter}\n[pounds per acre]'.capitalize())

ax.tick_params(bottom=True, top=True, left=True, right=False)
handles, labels = ax.get_legend_handles_labels()
rf.legend(ax, handles, labels, bbox_to_anchor=(1.2, 0))

fig.tight_layout()
if save_figures:
    fig_str = str(figure_count).rjust(2,'0')
    fig.savefig(f'../figures/{fig_str}_annual_{parameter}_load.png', dpi=150)
    figure_count += 1 

### Breakdown by River Basin

In [ ]:
baseline_loads = ambient_loads.sel(year=ambient_loads.year.dt.year.isin(baseline_years))
current_loads = supergage_loads.sel(year=supergage_loads.year.dt.year.isin(study_years))

current_loads.pint.to('ton').groupby('river').mean(dim='year').to_dataframe()

In [ ]:
# yield by river
baseline_yields = baseline_loads/network['drain_area_va']
current_yields = current_loads/network['drain_area_va']
current_yields.groupby('river').mean(dim='year').to_dataframe()
#baseline_yields.groupby('river').mean(dim='year').to_dataframe()

In [ ]:
#parameter = 'nitrate'
#fig, ax = plt.subplots(figsize=(fig_w, fig_h))
#
#mean = plot_by_basin(current_loads[parameter],
#              color='lightgreen',
#              ax=ax)
#
#ax.set_xticklabels([ label.get_text().replace(' at ','\nat ') for label in ax.get_xticklabels()], rotation=60, ha='right')
#ax.set_xlabel('Location'.capitalize())
##text = ax.set_ylabel(f'change in {parameter} load\n[percentage above baseline]'.capitalize())
#
#fig.tight_layout()

In [ ]:
parameter = 'nitrate nitrogen'
fig, ax = plt.subplots(figsize=(fig_w, fig_h))

plot_change_by_basin(baseline_loads[parameter].pint.to('tons'),
                     current_loads[parameter].pint.to('tons'),
                     color='lightgreen',
                     mode='load',
                     da=network['drain_area_va'],
                     ax=ax)


ax.set_xticklabels([label.get_text().replace(' at ','\nat ') for label in ax.get_xticklabels()], rotation=60, ha='right')
ax.set_xlabel('Location'.capitalize())
ax2 = percentage_scale(baseline_mean_load[parameter].pint.to('tons').values, ax, diff=True)
ax2.set_ylabel('[percentage of statewide baseline]'.capitalize())
text = ax.set_ylabel(f'change in {parameter} \n[tons]'.capitalize())
format_yaxis(ax)
fig.tight_layout()


if save_figures:
    fig_str = str(figure_count).rjust(2,'0')
    fig.savefig(f'../figures/{fig_str}_{parameter}_basin_load.png', dpi=150)
    figure_count += 1

In [ ]:
fig, ax = plt.subplots(figsize=(fig_w, fig_h))

plot_change_by_basin(baseline_loads[parameter], 
                     current_loads[parameter], 
                     color='lightgreen',
                     mode='yield',
                     da=network['drain_area_va'],
                     ax=ax)

ax.set_xticklabels([ label.get_text().replace(' at ','\nat ') for label in ax.get_xticklabels()], rotation=60, ha='right')
text = ax.set_ylabel(f'Change in {parameter}\n[pounds per acre]'.capitalize())
ax.set_xlabel('Location'.capitalize())
fig.tight_layout()

if save_figures:
    fig_str = str(figure_count).rjust(2,'0')
    fig.savefig(f'../figures/{fig_str}_{parameter}_basin_yield.png', dpi=150)
    figure_count += 1 

In [ ]:
parameter = 'total phosphorus'

fig, ax = plt.subplots(figsize=(fig_w, fig_h))

plot_change_by_basin(baseline_loads[parameter].pint.to('ton'),
                     current_loads[parameter].pint.to('ton'),
                     color='lavender',
                     mode='load',
                     da=network['drain_area_va'],
                     ax=ax)

ax2 = percentage_scale(baseline_mean_load[parameter].pint.to('tons').values, ax, diff=True)
ax2.set_ylabel('[percentage of statewide baseline]'.capitalize())

ax.set_xlabel('Location'.capitalize())
ax.set_xticklabels([ label.get_text().replace(' at ','\nat ') for label in ax.get_xticklabels()], rotation=60, ha='right')
text = ax.set_ylabel(f'change in {parameter}\n[tons]'.capitalize())
format_yaxis(ax)
fig.tight_layout()

if save_figures:
    fig_str = str(figure_count).rjust(2,'0')
    fig.savefig(f'../figures/{fig_str}_{parameter}_basin_load.png', dpi=150)
    figure_count += 1

In [ ]:
fig, ax = plt.subplots(figsize=(fig_w, fig_h))

plot_change_by_basin(baseline_loads[parameter],
                     current_loads[parameter],
                     color='lavender',
                     mode='yield',
                     da=network['drain_area_va'],
                     ax=ax)

ax.set_xticklabels([ label.get_text().replace(' at ','\nat ') for label in ax.get_xticklabels()], rotation=60, ha='right')
text = ax.set_ylabel(f'Change in {parameter}\n[pounds per acre]'.capitalize())
ax.set_xlabel('Location'.capitalize())
fig.tight_layout()


if save_figures:
    fig_str = str(figure_count).rjust(2,'0')
    fig.savefig(f'../figures/{fig_str}_{parameter}_basin_yield.png', dpi=150)
    figure_count += 1 

## Streamflow

In [ ]:
parameter = 'mean flow'
statewide_streamflow = ambient_loads[parameter].groupby('year').sum(dim='river').pint.to(flow_unit)
baseline_streamflow = statewide_streamflow.sel(year=statewide_streamflow.year.dt.year.isin(baseline_years)).mean()

#statewide_streamflow_p = statewide_streamflow / baseline_streamflow * 100 - 100

fig, ax = plt.subplots(figsize=(fig_w, fig_h))

running_average_plot(ds1=statewide_streamflow.assign_attrs({'label':'Illinois EPA Ambient'}),
                     period1=baseline_years,
                     period2=study_years,
                     loc='upper left',
                     ax=ax)


ax2 = percentage_scale(baseline_streamflow.values, ax)

ax2.set_ylabel('[percentage above baseline]'.capitalize())
#text = ax.set_ylabel(f'{parameter}\n[cubic meters per second]'.capitalize())

ax.tick_params(bottom=True, top=True, left=True, right=False)
handles, labels = ax.get_legend_handles_labels()
rf.legend(ax, handles, labels, bbox_to_anchor=(1.25, 0))


text = ax.set_ylabel(f'statewide streamflow\n[{flow_unit}]'.capitalize())
format_yaxis(ax)
fig.tight_layout()

if save_figures:
    fig_str = str(figure_count).rjust(2,'0')
    fig.savefig(f'../figures/{fig_str}_annual_streamflow.png', dpi=150)
    figure_count += 1 

Change in streamflow yields by basin.

In [ ]:
# TODO flawed logic
# select streamflow for baseline and current timeperiods
parameter='mean flow'
flow = ambient_loads[parameter]
pflow = flow/flow.mean(dim='year') * 100
baseline_streamflow = pflow.sel(year=pflow.year.dt.year.isin(baseline_years))
current_streamflow = pflow.sel(year=pflow.year.dt.year.isin(study_years))


#statewide_streamflow_p = statewide_streamflow / baseline_streamflow * 100 - 100
fig, ax = plt.subplots(figsize=(fig_w, fig_h))

plot_change_by_basin(baseline_streamflow,
                     current_streamflow,
                     color='powderblue',
                     ax=ax,
                     da=network['drain_area_va'],
                     mode='load')

#percentage_scale(baseline_mean_flow.mean(), ax)
ax.set_xticklabels([ label.get_text().replace(' at ','\nat ') for label in ax.get_xticklabels()], rotation=60, ha='right')
#text = ax.set_ylabel(f'Change in streamflow\n[percentage above baseline]'.capitalize())
ax.set_xlabel('Location'.capitalize())
fig.tight_layout()

#if save_figures:
#    fig_str = str(figure_count).rjust(2,'0')
#    fig.savefig(f'../figures/{fig_str}_streamflow_basin.png', dpi=150)
#    figure_count += 1

## Flow-adjusted loads
Flow-adjusted nitrate loads by basin.

In [ ]:
# compute flow adjustment factore
parameter = 'mean flow'
flow = ambient_loads[parameter]
pflow = flow/flow.mean(dim='year')

In [ ]:
parameter = 'nitrate nitrogen'
fig, ax = plt.subplots(figsize=(fig_w, fig_h))

total_streamflow = ambient_loads['mean flow'].sum(dim='river')

plot_change_by_basin(baseline_loads[parameter].pint.to('ton')/pflow,
                     current_loads[parameter].pint.to('ton')/pflow,
                     color='lightgreen',
                     ax=ax,
                     da=network['drain_area_va'],
                     mode='load')

ax.set_xticklabels([ label.get_text().replace(' at ','\nat ') for label in ax.get_xticklabels()], rotation=60, ha='right')

text = ax.set_ylabel(f'flow-adjusted change in {parameter}\n[tons]'.capitalize())
#text = ax.set_ylabel(f'Change in flow-adjusted {parameter} load\n[percentage above baseline]'.capitalize())
ax.set_xlabel('Location'.capitalize())


ax2 = percentage_scale(baseline_mean_load[parameter].pint.to('tons').values, ax, diff=True)
ax2.set_ylabel('[percentage of statewide baseline]'.capitalize())

format_yaxis(ax)
fig.tight_layout()

if save_figures:
    fig_str = str(figure_count).rjust(2,'0')
    fig.savefig(f'../figures/{fig_str}_{parameter}_basin_fa_load.png', dpi=150)
    figure_count += 1

Flow-adjusted phosphorus load by basin

In [ ]:
parameter = 'total phosphorus'
fig, ax = plt.subplots(figsize=(fig_w, fig_h))


plot_change_by_basin(baseline_loads[parameter].pint.to('ton')/pflow, #total_streamflow,
                     current_loads[parameter].pint.to('ton')/pflow, #total_streamflow,
                     color='lavender',
                     ax=ax,
                     da=network['drain_area_va'],
                     mode='load')

ax.set_xticklabels([ label.get_text().replace(' at ','\nat ') for label in ax.get_xticklabels()], rotation=60, ha='right')
text = ax.set_ylabel(f'flow-adjusted change in {parameter}\n[tons]'.capitalize())
ax.set_xlabel('Location'.capitalize())

ax2 = percentage_scale(baseline_mean_load[parameter].pint.to('tons').values, ax, diff=True)
ax2.set_ylabel('[percentage of statewide baseline]'.capitalize())

format_yaxis(ax)
fig.tight_layout()

if save_figures:
    fig_str = str(figure_count).rjust(2,'0')
    fig.savefig(f'../figures/{fig_str}_{parameter}_basin_fa_load.png', dpi=150)
    figure_count += 1

## Summary
1. Nitrate load increased 7.5%, primarily from the Rock River
1. Phosphorus load increased 40%, primarily from the Illinois, Kaskaskia, and Little Wabash Rivers
1. Streamflow increased 23% statewide
1. Adjusting for streamflow, nitrate loads declined 10%
1. Adjusting for streamflow, phosphorus load increased 15%.


## References
U.S. Geological Survey, 2022, National Water Information System data available on the World Wide Web (USGS Water Data for the Nation), accessed [July 25, 2022], at URL [http://waterdata.usgs.gov/nwis/].